# Data Modelisation

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from gensim.utils import ClippedCorpus
import gensim
import tqdm
from gensim.models import Phrases
from gensim.corpora import Dictionary
from nltk.tokenize import RegexpTokenizer
from gensim.models import LdaModel, CoherenceModel, LdaMulticore, TfidfModel, Nmf
from nlp_module import remove_stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

## 1) Data preparation

In [2]:
data = pd.read_csv("datasets/posts_clean.csv")
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,<python><web-services>
1,415344,log implementation prefer,I implement log class c try decide I curious k...,<debugging><language-agnostic><logging>
2,414981,directly modify list element,I struct struct map public int size public map...,<c#><.net>


In [3]:
# remove < and > around Tags
data["Tags"] = data["Tags"].replace({"<" : " "}, regex=True)
data["Tags"] = data["Tags"].replace({">" : " "}, regex=True)

In [4]:
data["Body"] = data["Body"].str.lower()
data["Body"] = data["Body"].apply(remove_stopwords)

In [5]:
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,use python year little experience python web p...,python web-services
1,415344,log implementation prefer,implement log class c try decide curious know ...,debugging language-agnostic logging
2,414981,directly modify list element,struct struct map public int size public map i...,c# .net


In [6]:
docs = data["Body"].to_list()

In [7]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [8]:
# Compute bigrams.
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [9]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dct = Dictionary(docs)
dct.filter_extremes(no_below=20, no_above=0.5)

In [10]:
# Bag-of-words representation of the documents.
corpus = [dct.doc2bow(doc) for doc in docs]

In [18]:
model = TfidfModel(corpus)
corpus = model[corpus]

In [21]:
print('Number of unique tokens: %d' % len(dct))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 25556
Number of documents: 164598


## 2) Topic Modelling

In [22]:
# Set training parameters.
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

# Make a index to word dictionary.
temp = dct[0]  # This is only to "load" the dictionary.
id2word = dct.id2token

### a) Non Negative Matrix Factorization (NMF)

In [23]:
# Create a list of the topic numbers we want to try
topic_nums = list(np.arange(2, 10 + 1, 1))

# Run the nmf model and calculate the coherence score
# for each number of topics
coherence_scores = []

if 1 == 1:
    pbar = tqdm.tqdm(total=200)

    for num in topic_nums:
        nmf = Nmf(
            corpus=corpus,
            num_topics=num,
            id2word=id2word,
            chunksize=chunksize,
            passes=passes,
            kappa=.1,
            minimum_probability=0.01,
            w_max_iter=300,
            w_stop_condition=0.0001,
            h_max_iter=100,
            h_stop_condition=0.001,
            eval_every=eval_every,
            normalize=True,
            random_state=42)
    
    # Run the coherence model to get the score
        cm = CoherenceModel(
            model=nmf,
            texts=docs,
            dictionary=dct,
            coherence='c_v')
    
        coherence_scores.append(round(cm.get_coherence(), 5))
        
        pbar.update(1)

pbar.close()

# Get the number of topics with the highest coherence score
scores = list(zip(topic_nums, coherence_scores))
best_num_topics = sorted(scores, key=itemgetter(1), reverse=True)[0][0]

print(best_num_topics)

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
nmf = Nmf(corpus,
          id2word = id2word,
          chunksize = 2000,
          num_topics=9,
          kappa=0.1)

In [26]:
coherence_model_nmf = CoherenceModel(model=nmf, texts=docs, dictionary=dct, coherence='c_v')
coherence_nmf = coherence_model_nmf.get_coherence()
print('Coherence Score: ', coherence_nmf)

Coherence Score:  0.55068189032469


### b) Latent Dirichlet Allocation (LDA)

In [17]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = LdaMulticore(corpus=corpus,
                        id2word = id2word,
                        chunksize=chunksize,
                        alpha=a,
                        eta=b,
                        iterations=iterations,
                        num_topics=k,
                        passes=passes,
                        eval_every=eval_every)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [19]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dct, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('datasets/lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [38:30:52<00:00, 256.76s/it]


In [24]:
pd.DataFrame(model_results).sort_values("Coherence", ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
491,100% Corpus,9,0.61,0.31,0.640743
535,100% Corpus,10,asymmetric,0.01,0.636625
195,75% Corpus,8,0.91,0.01,0.629944
252,75% Corpus,10,0.61,0.61,0.629695
221,75% Corpus,9,0.61,0.31,0.626676
...,...,...,...,...,...
15,75% Corpus,2,0.91,0.01,0.440731
290,100% Corpus,2,symmetric,0.01,0.439272
280,100% Corpus,2,0.61,0.01,0.434665
279,100% Corpus,2,0.31,symmetric,0.434100


In [28]:
lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=9, 
                        random_state=42,
                        chunksize=chunksize,
                        passes=passes,
                        iterations=iterations,
                        alpha=0.61,
                        eta=0.31,
                        eval_every=eval_every)

In [29]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5950434040553455


In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
lda_vis = gensimvis.prepare(lda_model, corpus, dct)
lda_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.090539 -0.063715       1        1  14.053951
1      0.122086 -0.087981       2        1  11.795424
4      0.206204  0.057128       3        1  11.385928
7     -0.198941 -0.162112       4        1  11.372348
3     -0.051175  0.277182       5        1  10.696222
5      0.023420  0.005149       6        1  10.578801
0     -0.173368  0.005562       7        1  10.277779
2      0.068545 -0.097639       8        1  10.138149
6     -0.087311  0.066426       9        1   9.701398, topic_info=        Term         Freq        Total Category  logprob  loglift
736      div  1629.000000  1629.000000  Default  30.0000  30.0000
75       int  2163.000000  2163.000000  Default  29.0000  29.0000
40     class  1747.000000  1747.000000  Default  28.0000  28.0000
372    table  1298.000000  1298.000000  Default  27.0000  27.0000
80    public  1475.000000  1475.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
78       new   333.301047  1536.645517   Topic9  -5.4921   0.8046
546     text   295.231388  1319.146700   Topic9  -5.6134   0.8359
833   window   246.251395   780.721075   Topic9  -5.7948   1.1790
2153   child   229.101514   509.245222   Topic9  -5.8670   1.5341
0        add   213.021266  1135.630043   Topic9  -5.9397   0.6593

[622 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
7930      6  0.969962  _post
3224      1  0.021333    abc
3224      2  0.028444    abc
3224      3  0.120889    abc
3224      4  0.003556    abc
...     ...       ...    ...
31        7  0.003076   year
31        8  0.003076   year
31        9  0.003076   year
8441      1  0.945614     za
8441      6  0.023348     za

[2599 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 5, 8, 4, 6, 1, 3, 7])